In [1]:
pwd

'/home/ec2-user/SageMaker'

In [4]:
input_text=['Railway', 'Contact/Call Center', 'Power generation contact angle', 'Blood pressure equipment', 'Other',
 'Health, Location & Asset Tracking, Home, Other', 'Washing machines', 'Rotary applications, such as pumps, rollers, mixers, fan speed measurement, transmission, spindles, gear reducer RPM and process speed',
 'Smart Cities, Energy and Utilities, Home, Facilities Management, Smart Building', 'Remote Collaboration',
 'Hydraulic pump motors in material handling, agriculture, and construction machines: May be used to help control power delivered by the hydraulic pump motor',
 'Industrial controls', 'Unified Communications', 'Smart Cities, Smart Industries, Daily Life', 'Remote Monitoring',
 'Medical: Anesthesia delivery machines, blood analyzers, chemistry analyzers, gas chromatography, oxygen concentrators, and ventilators',
 'Smart Agriculture, Smart Cities, Energy and Utilities, Health, Smart Industries, Transportation, Home, Facilities Management',
 'Health, Smart Industries, Transportation', 'Handheld air quality monitors', 'Energy and Utilities, Home, Facilities Management',
 'Infusion pumps\xa0', 'Pneumatic controls', 'Steering angle', 'Wafer handling: May be used to monitor if parts are properly placed.']


Master_Groups=['Smart Cities', 'Asset Tracking', 'Healthcare', 'Smart Agriculture', 'Energy and Utilities', 'Smart Industries',
 'Transportation', 'Telecommunications', 'Smart Homes', 'Smart Building', 'Power Generation', 'Automotive', 'Meteorology']



import pandas as pd
import numpy as np
import itertools

listoflists=[i.split(',') for i in input_text]
merged = list(itertools.chain.from_iterable(listoflists))
alll=[x.strip() for x in merged]

unique_text=list(set(alll))
unique_text

print('loading Model......')
#!pip install git+https://github.com/AndriyMulyar/semantic-text-similarity
from semantic_text_similarity.models import WebBertSimilarity
from semantic_text_similarity.models import ClinicalBertSimilarity

web_model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

clinical_model = ClinicalBertSimilarity(device='cuda', batch_size=10) #defaults to GPU prediction
#web_model.predict([("Transportation ","Surgery ")])
print('Predicting Ip_text......')
def Model_prediction(unique_input_item,master_category):
    out=[]
    for i in unique_input_item:
        bst=[]
        for mstr in master_category:
            val=web_model.predict([(mstr,i)])
            #print(val)
            bst.append(val.tolist())
        d=dict(zip(master_category,bst))
        category=list(dict(sorted(d.items(), key=lambda x: x[1], reverse=True)[:1]).keys())
        out.append(category)
    
    return out
out=Model_prediction(unique_text,Master_Groups)

dfout=pd.DataFrame([unique_text,out]).T
dfout=dfout.rename(columns={0:'Input',1:'Application_Type'})
dfout.head()

dfout['Application_Type']=dfout['Application_Type'].astype(str).str.replace('\[|\]|\'', '')
by_group=dfout.groupby(['Application_Type'], as_index=False, sort=False).agg(','.join)
ip_terms=[x.split(',') for x in by_group['Input']]
master_mapping=dict(zip(list(by_group['Application_Type']),ip_terms))
master_mapping

print('Mapping IP & Output.......')

def Mapping_input_and_predicted_output(Ip_listoflist,Master_cat):
    strp=[]
    for one in np.arange(len(Ip_listoflist)):
        val=[i.strip() for i in Ip_listoflist[one]]
        strp.append(val)
    #print(strp)
    oop=[]
    for item in strp:
        op=[]
        for key, value in Master_cat.items():
            for i in value:
                if  i in item:
                    op.append(key)
        oop.append(op)
    len(oop)
    #print(oop)
    return strp,oop

input_string,master_grop=Mapping_input_and_predicted_output(listoflists,master_mapping)
input_string

result=[list(i) for i in [set(x) for x in master_grop]]

finaldf=pd.DataFrame([input_string,result]).T
finaldf[1]=finaldf[1].astype(str).str.replace('\[|\]|\'', '')
Output=finaldf.rename(columns={0:'Input',1:'Mapped_Master_Type'})
#Output.to_excel('output.xlsx')
Output.head(10)

loading Model......
Predicting Ip_text......
Mapping IP & Output.......


,Input,Mapped_Master_Type
0,[Railway],Transportation
1,[Contact/Call Center],Telecommunications
2,[Power generation contact angle],Power Generation
3,[Blood pressure equipment],Healthcare
4,[Other],Telecommunications
5,"[Health, Location & Asset Tracking, Home, Other]","Healthcare, Smart Homes, Asset Tracking, Telec..."
6,[Washing machines],Healthcare
7,"[Rotary applications, such as pumps, rollers, ...","Automotive, Power Generation, Asset Tracking, ..."
8,"[Smart Cities, Energy and Utilities, Home, Fac...","Smart Cities, Asset Tracking, Energy and Utili..."
9,[Remote Collaboration],Telecommunications


In [1]:
input_text=['Railway',
 'Contact/Call Center',
 'Power generation contact angle',
 'Blood pressure equipment',
 'Other',
 'Health, Location & Asset Tracking, Home, Other',
 'Washing machines',
 'Rotary applications, such as pumps, rollers, mixers, fan speed measurement, transmission, spindles, gear reducer RPM and process speed',
 'Smart Cities, Energy and Utilities, Home, Facilities Management, Smart Building',
 'Remote Collaboration',
 'Hydraulic pump motors in material handling, agriculture, and construction machines: May be used to help control power delivered by the hydraulic pump motor',
 'Industrial controls',
 'Unified Communications',
 'Smart Cities, Smart Industries, Daily Life',
 'Remote Monitoring',
 'Medical: Anesthesia delivery machines, blood analyzers, chemistry analyzers, gas chromatography, oxygen concentrators, and ventilators',
 'Smart Agriculture, Smart Cities, Energy and Utilities, Health, Smart Industries, Transportation, Home, Facilities Management',
 'Health, Smart Industries, Transportation',
 'Handheld air quality monitors',
 'Energy and Utilities, Home, Facilities Management',
 'Infusion pumps\xa0',
 'Pneumatic controls',
 'Steering angle',
 'Wafer handling: May be used to monitor if parts are properly placed.']

In [2]:
Master_Groups=['Smart Cities', 'Asset Tracking', 'Healthcare', 'Smart Agriculture', 'Energy and Utilities', 'Smart Industries',
 'Transportation', 'Telecommunications', 'Smart Homes', 'Smart Building', 'Power Generation', 'Automotive', 'Meteorology']

In [3]:
import pandas as pd
import numpy as np
import itertools

listoflists=[i.split(',') for i in input_text]
merged = list(itertools.chain.from_iterable(listoflists))
alll=[x.strip() for x in merged]

unique_text=list(set(alll))
unique_text[:10]


['Rotary applications',
 'Medical: Anesthesia delivery machines',
 'Infusion pumps',
 'Daily Life',
 'agriculture',
 'Transportation',
 'Steering angle',
 'fan speed measurement',
 'Power generation contact angle',
 'and construction machines: May be used to help control power delivered by the hydraulic pump motor']

##  Bert Model

In [4]:
#!pip install git+https://github.com/AndriyMulyar/semantic-text-similarity
from semantic_text_similarity.models import WebBertSimilarity
#from semantic_text_similarity.models import ClinicalBertSimilarity

web_model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

#clinical_model = ClinicalBertSimilarity(device='cuda', batch_size=10) #defaults to GPU prediction
#web_model.predict([("Transportation ","Surgery ")])

In [5]:
def Model_prediction(unique_input_item,master_category):
    out=[]
    for i in unique_input_item:
        bst=[]
        for mstr in master_category:
            val=web_model.predict([(mstr,i)])
            #print(val)
            bst.append(val.tolist())
        d=dict(zip(master_category,bst))
        category=list(dict(sorted(d.items(), key=lambda x: x[1], reverse=True)[:1]).keys())
        out.append(category)
    
    return out
out=Model_prediction(unique_text,Master_Groups)

dfout=pd.DataFrame([unique_text,out]).T
dfout=dfout.rename(columns={0:'Input',1:'Application_Type'})
dfout.head()

,Input,Application_Type
0,Rotary applications,[Power Generation]
1,Medical: Anesthesia delivery machines,[Healthcare]
2,Infusion pumps,[Healthcare]
3,Daily Life,[Healthcare]
4,agriculture,[Smart Agriculture]


In [7]:
dfout['Application_Type']=dfout['Application_Type'].astype(str).str.replace('\[|\]|\'', '')
by_group=dfout.groupby(['Application_Type'], as_index=False, sort=False).agg(','.join)
ip_terms=[x.split(',') for x in by_group['Input']]
master_mapping=dict(zip(list(by_group['Application_Type']),ip_terms))
master_mapping

{'Power Generation': ['Rotary applications',
  'Power generation contact angle',
  'and construction machines: May be used to help control power delivered by the hydraulic pump motor',
  'oxygen concentrators',
  'transmission'],
 'Healthcare': ['Medical: Anesthesia delivery machines',
  'Infusion pumps',
  'Daily Life',
  'Handheld air quality monitors',
  'Washing machines',
  'Blood pressure equipment',
  'Health',
  'Pneumatic controls'],
 'Smart Agriculture': ['agriculture', 'Smart Agriculture'],
 'Transportation': ['Transportation', 'Railway'],
 'Automotive': ['Steering angle',
  'fan speed measurement',
  'Hydraulic pump motors in material handling',
  'gear reducer RPM and process speed'],
 'Smart Homes': ['Home'],
 'Smart Building': ['Smart Building'],
 'Telecommunications': ['Unified Communications',
  'Remote Collaboration',
  'Wafer handling: May be used to monitor if parts are properly placed.',
  'Contact/Call Center',
  'Other'],
 'Meteorology': ['chemistry analyzers'],


In [10]:
def Mapping_input_and_predicted_output(Ip_listoflist,Master_cat):
    strp=[]
    for one in np.arange(len(Ip_listoflist)):
        val=[i.strip() for i in Ip_listoflist[one]]
        strp.append(val)
    #print(strp)
    oop=[]
    for item in strp:
        op=[]
        for key, value in Master_cat.items():
            for i in value:
                if  i in item:
                    op.append(key)
        oop.append(op)
    len(oop)
    #print(oop)
    return strp,oop

input_string,master_grop=Mapping_input_and_predicted_output(listoflists,master_mapping)
input_string[:6]

[['Railway'],
 ['Contact/Call Center'],
 ['Power generation contact angle'],
 ['Blood pressure equipment'],
 ['Other'],
 ['Health', 'Location & Asset Tracking', 'Home', 'Other']]

In [9]:
result=[list(i) for i in [set(x) for x in master_grop]]

finaldf=pd.DataFrame([input_string,result]).T
finaldf[1]=finaldf[1].astype(str).str.replace('\[|\]|\'', '')
Output=finaldf.rename(columns={0:'Input',1:'Mapped_Master_Type'})
#Output.to_excel('output.xlsx')
Output.head(10)

,Input,Mapped_Master_Type
0,[Railway],Transportation
1,[Contact/Call Center],Telecommunications
2,[Power generation contact angle],Power Generation
3,[Blood pressure equipment],Healthcare
4,[Other],Telecommunications
5,"[Health, Location & Asset Tracking, Home, Other]","Asset Tracking, Healthcare, Smart Homes, Telec..."
6,[Washing machines],Healthcare
7,"[Rotary applications, such as pumps, rollers, ...","Power Generation, Automotive, Asset Tracking, ..."
8,"[Smart Cities, Energy and Utilities, Home, Fac...","Asset Tracking, Smart Homes, Energy and Utilit..."
9,[Remote Collaboration],Telecommunications
